In [1]:

# import yaml

# from bokeh.layouts import column
# from bokeh.models import ColumnDataSource, Slider
# from bokeh.plotting import figure
# from bokeh.themes import Theme
# from bokeh.io import show, output_notebook

# from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature

# import os
# import urllib

# output_notebook()

Loading BokehJS ...

In [6]:
import datetime
import urllib
from os.path import dirname, join
import os

import pandas as pd
#from scipy.signal import savgol_filter

from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, DataRange1d, Select, RadioButtonGroup
from bokeh.palettes import Blues4
from bokeh.plotting import figure
from bokeh.io import show, output_notebook

output_notebook()

Loading BokehJS ...

In [2]:
# def bkapp(doc):
#     df = sea_surface_temperature.copy()
#     source = ColumnDataSource(data=df)

#     plot = figure(x_axis_type='datetime', y_range=(0, 25),
#                   y_axis_label='Temperature (Celsius)',
#                   title="Sea Surface Temperature at 43.18, -70.43")
#     plot.line('time', 'temperature', source=source)

#     def callback(attr, old, new):
#         if new == 0:
#             data = df
#         else:
#             data = df.rolling('{0}D'.format(new)).mean()
#         source.data = ColumnDataSource.from_df(data)

#     slider = Slider(start=0, end=30, value=0, step=1, title="Smoothing by N Days")
#     slider.on_change('value', callback)

#     doc.add_root(column(slider, plot))

#     doc.theme = Theme(json=yaml.load("""
#         attrs:
#             Figure:
#                 background_fill_color: "#DDDDDD"
#                 outline_line_color: white
#                 toolbar_location: above
#                 height: 500
#                 width: 800
#             Grid:
#                 grid_line_dash: [6, 4]
#                 grid_line_color: white
#     """, Loader=yaml.FullLoader))

In [7]:
def remote_jupyter_proxy_url(port):
    """
    Callable to configure Bokeh's show method when a proxy must be
    configured.

    If port is None we're asking about the URL
    for the origin header.
    """
    #base_url = os.environ['EXTERNAL_URL']
    base_url = 'https://jupyterhub.cuahsi.org'
    host = urllib.parse.urlparse(base_url).netloc
    
    # If port is None we're asking for the URL origin
    # so return the public hostname.
    if port is None:
        return host

    service_url_path = os.environ['JUPYTERHUB_SERVICE_PREFIX']
    proxy_url_path = 'proxy/%d' % port

    user_url = urllib.parse.urljoin(base_url, service_url_path)
    full_url = urllib.parse.urljoin(user_url, proxy_url_path)
    return full_url

In [4]:
# show(bkapp, notebook_url=remote_jupyter_proxy_url)

In [58]:

def bkapp(doc):
    plot = figure(x_axis_type="datetime",
                  plot_width=800)
    source = ColumnDataSource()

           
    mode = 'cumulative'
    style= 'line'
    agg = 'daily'

    

    
    def update_plot(attrname, old, new):
        
        select_dataset_label = data_select.labels[data_select.active]
        
        plot.title.text = select_dataset_label
        
        

        #df = pd.read_csv('dat/covid-deaths.tsv',
        df = pd.read_csv('dat/covid-deaths.tsv',
                         delimiter='\t')
    
    
        state = state_select.value
        plot.title.text = f"{plot_title}{state}"
        source.data = dict(x=dates, y=df[state])
        update_style(None, None, style_select.value)
        update_mode(None, None, mode_select.value)
        update_agg(None, None, agg_select.value)

    def update_style(attrname, old, new):
        style = new.lower()
        if style == 'line':
            line.visible = True
            vbar.visible = False
        if style == 'bar':
            line.visible = False
            vbar.visible = True

    def update_mode(attrname, old, new):
        agg = new.lower()
        state = state_select.value
        if agg == 'cumulative':
            # plot raw cumulative data
            source.data = dict(x=dates, y=df[state])
        if agg == 'incremental':
            # compute incremental
            diffs = df[state].diff()
            diffs[diffs < 0] = 0
            source.data = dict(x=dates,
                               y=diffs)
        if agg == 'percent change':
            # compute percent change
            pct = df[state].diff().pct_change()
            pct = pct.fillna(0)
            source.data = dict(x=dates,
                               y=pct)

    def update_agg(attrname, old, new):
        # create new dataframe from current x,y data
        # then perform date agg
#        df = pd.DataFrame(source.data)
#        df = df.set_index('x')

        agg = agg_select.value
        state = state_select.value

        
        if agg == 'weekly':
            df = df.groupby(pd.TimeGrouper('W')).sum().dropna()
        if agg == 'daily':
            df = df.groupby(pd.TimeGrouper('D')).sum().dropna()

        source.data = dict(x=df.index, y=df.y)



    def update_data(attrname, old, new):
        global df, plot_title
        selected = data_select.labels[new]
        if selected == 'Confirmed':
            df = pd.read_csv('dat/covid-confirmed.tsv',
                             delimiter='\t')
            plot_title = 'COVID Confirmed: ' 
        else:
            df = pd.read_csv('dat/covid-deaths.tsv',
                             delimiter='\t')
            plot_title = 'COVID Deceased: ' 
        update_plot('', '', '')
    
 

    state = 'Massachusetts'
    states = df.columns[1:]
    state_select = Select(value=state,
                          title='State',
                          options=sorted(states))
    mode_select = Select(value='Cumulative',
                         title='Mode',
                         options=['Cumulative',
                                  'Incremental',
                                  'Percent Change'])
    agg_select = Select(value='Daily',
                        title='Aggregation',
                        options=['Daily',
                                 'Weekly',
                                 '7-Day Ave'])
    style_select = Select(value='Line',
                            title='Style',
                            options=['Line',
                                     'Bar'])

    dates = pd.to_datetime(df.date, format='%Y%m%d')
    source = ColumnDataSource(data=dict(x=dates,
                                        y=df[state]))

    line = plot.line('x', 'y',
                     source=source,
                     line_width=3,
                     line_alpha=0.6)
    vbar = plot.vbar(x='x', top='y', width=0.2,
                     source=source)
    vbar.visible = False


    # fixed attributes
    plot_title = 'COVID Deceased: ' 
    plot.xaxis.axis_label = None
    plot.yaxis.axis_label = "Count"
    plot.axis.axis_label_text_font_style = "bold"
    plot.x_range = DataRange1d(range_padding=0.0)
    plot.grid.grid_line_alpha = 0.3

    data_select = RadioButtonGroup(labels=['Confirmed', 'Deceased'], active=1)

    state_select.on_change('value', update_plot)
    style_select.on_change('value', update_style)
    mode_select.on_change('value', update_mode)
    agg_select.on_change('value', update_agg)
    data_select.on_change('active', update_data)

    controls = column(data_select,
                      state_select,
                      mode_select,
                      style_select,
                      agg_select)


    doc.add_root(row(plot, controls))


    #    state = state_select.value
    #    plot.title.text = f"{plot_title}{state}"




In [59]:
#os.environ['BOKEH_LOG_LEVEL'] = 'info'
handle = show(bkapp, notebook_url=remote_jupyter_proxy_url)  


In [ ]:
handle

In [ ]:
df = pd.read_csv('dat/covid-confirmed.tsv','\t')

In [ ]:
max(source.data['y'])

In [ ]:
update_plot('','','')

In [ ]:
from bokeh.io import push_notebook

In [ ]:
push_notebook(handle=handle)

In [ ]:

df = pd.read_csv('dat/covid-deaths.tsv',
                     delimiter='\t')

In [ ]:

def bkapp(doc):

    df = sea_surface_temperature.copy()
    source = ColumnDataSource(data=df)

    plot = figure(x_axis_type='datetime', y_range=(0, 25),
                  y_axis_label='Temperature (Celsius)',
                  title="Sea Surface Temperature at 43.18, -70.43")
    plot.line('time', 'temperature', source=source)

    def callback(attr, old, new):
        if new == 0:
            data = df
        else:
            data = df.rolling('{0}D'.format(new)).mean()
        source.data = ColumnDataSource.from_df(data)

    slider = Slider(start=0, end=30, value=0, step=1, title="Smoothing by N Days")
    slider.on_change('value', callback)

    doc.add_root(row(slider, plot))

    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                background_fill_color: "#DDDDDD"
                outline_line_color: white
                toolbar_location: above
                height: 500
                width: 800
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: white
    """, Loader=yaml.FullLoader))

In [ ]:
show(bkapp, notebook_url=remote_jupyter_proxy_url) 

In [ ]:


df = sea_surface_temperature.copy()
source = ColumnDataSource(data=df)

plot = figure(x_axis_type='datetime', y_range=(0, 25),
              y_axis_label='Temperature (Celsius)',
              title="Sea Surface Temperature at 43.18, -70.43")
plot.line('time', 'temperature', source=source)

def callback(attr, old, new):
    if new == 0:
        data = df
    else:
        data = df.rolling('{0}D'.format(new)).mean()
    source.data = ColumnDataSource.from_df(data)

slider = Slider(start=0, end=30, value=0, step=1, title="Smoothing by N Days")
slider.on_change('value', callback)

cd = curdoc()
cd.add_root(row(slider, plot))

cd.theme = Theme(json=yaml.load("""
    attrs:
        Figure:
            background_fill_color: "#DDDDDD"
            outline_line_color: white
            toolbar_location: above
            height: 500
            width: 800
        Grid:
            grid_line_dash: [6, 4]
            grid_line_color: white
""", Loader=yaml.FullLoader))

In [ ]:
show(plot, notebook_url=remote_jupyter_proxy_url, notebook_handle=True) 